In [1]:
from deeppavlov.core.models.component import Component
from deeppavlov.models.classifiers.tokens_matcher.tokens_matcher import TokensMatcher

/home/yoptar/reps/DeepPavlov/venv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
[nltk_data] Downloading package punkt to /home/yoptar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/yoptar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/yoptar/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /home/yoptar/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!
2018-06-18 18:06:55.74 DEBUG in 'gensim.models.doc2vec'['doc2vec'] at line 73: Fast version

In [2]:
class SimpleSkill(Component):
    @staticmethod
    def _confidence(utterances_batch):
        return [1 for _ in utterances_batch]  # by default always be confident
    
    def __init__(self, response, confidence_fn=None, *args, **kwargs):
        self.response = response
        if confidence_fn is not None:
            self._confidence = confidence_fn
    
    def __call__(self, utterances_batch, history_batch, states_batch):
        tokens = [u.lower().split() for u in utterances_batch]
        return [self.response for _ in utterances_batch], self._confidence(tokens)

In [3]:
hello = SimpleSkill('Hello world!', confidence_fn=TokensMatcher(["hi", "hello", "good day"]))
bye = SimpleSkill('Goodbye world!', confidence_fn=TokensMatcher(["bye", "chao", "see you"]))
other = SimpleSkill("I don't understand, sorry")

In [4]:
from deeppavlov.core.agent.agent import Agent, HighestConfidenceSelector

agent = Agent([hello, bye, other], skills_selector=HighestConfidenceSelector())

In [5]:
agent(['Hello', 'Bye', 'Or not'])

['Hello world!', 'Goodbye world!', "I don't understand, sorry"]